In [1]:
from __future__ import print_function, division
import math

import numpy as np
import pandas as pd
import torch

In [158]:
path = "C:\\Users\\coco1\\PycharmProjects\\PatternRecignitionHM01\\01\\data\\lung-cancer.data"
raw_data = pd.read_csv(path,header=None, dtype=float)

In [196]:
amount = len(raw_data)
train_amount = int(amount * 0.7)
test_amount = amount - train_amount
def PCA(data, k=2):
 # preprocess the data
 X = data
 X_mean = torch.mean(X,0)
 X = X - X_mean.expand_as(X)
 # svd
 U,S,V = torch.svd(torch.t(X))
 return torch.mm(X,U[:,:k])
train_set = raw_data.sample(frac=1).reset_index(drop=True).values

In [197]:
# define the LDF (Linear Discriminant Analysis)
# Under LDA we assume that the density for X, given every class k is following a Gaussian distribution. 
# LDA computes “discriminant scores” for each observation to classify what response variable class it is in 
# (i.e. default or not default). 
# These scores are obtained by finding linear combinations of the independent variables. 
# For a single predictor variable X = x

train_tensor = torch.tensor(train_set[:, 1:], dtype=torch.float)
data_iter = PCA(train_tensor, 20)
label_iter = train_set[:,0]
        

In [219]:
train_data = torch.tensor(np.array(data_iter[0:20, :]), dtype=torch.float)
test_data= torch.tensor(np.array(data_iter[20:, :]), dtype=torch.float)
train_label = torch.tensor(np.array(label_iter[0:20]), dtype=torch.int)
test_label = torch.tensor(np.array(label_iter[20:]), dtype=torch.int)

In [220]:
type1_train_data = []
type2_train_data = []
type3_train_data = []
for i in range(train_label.size()[0]) :
    if train_label[i] == 1 : type1_train_data.append(train_data[i])
    if train_label[i] == 2 : type2_train_data.append(train_data[i])
    if train_label[i] == 3 : type3_train_data.append(train_data[i])

In [226]:
# get the mu
def get_lda_norm_attribute (train, all_size) : 
    init_vec = torch.zeros(train[0].size()[0], dtype=torch.float)
    for vec in train :
        init_vec += vec
    return init_vec / len(train), len(train) / all_size
def get_lda_cor_matrix (train, K, mu) :
    print(train.size())
    zeros = torch.zeros((train.size()[1],train.size()[1]))
    print(zeros.size())
    print(mu.size())
    size = train.size()[0]
    for vec in train : 
        print((vec - mu).size())
        zeros += (vec - mu).view(20, 1).mm((vec - mu).view(1,20))
    return zeros / (size - K)
def get_lda_cor_inv (train, K, mu) :
    return np.linalg.inv(get_lda_cor_matrix(train, K, mu))

In [227]:
#combining together
all_size = train_data.size()[0]
mu_1, pi_1 = get_lda_norm_attribute(type1_train_data, all_size)
mu_2, pi_2 = get_lda_norm_attribute(type2_train_data, all_size)
mu_3, pi_3 = get_lda_norm_attribute(type3_train_data, all_size)
cor_ma = get_lda_cor_matrix(train_data, 3, mu_1)
cor_ma_inv = get_lda_cor_inv(train_data, 3, mu_1)

torch.Size([20, 20])
torch.Size([20, 20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20, 20])
torch.Size([20, 20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])
torch.Size([20])


In [244]:
def ldf(x, mu, pi, cor_inv) :
    return -1 * (x.view(1,20).mm(cor_inv)).mm(x.view(20,1))  +  \
           2 * (mu.view(1,20).mm(cor_inv)).mm(x.view(20,1)) - \
           (mu.view(1,20).mm(cor_inv)).mm(mu.view(20,1)) + 2 * math.log2(pi)
right = 0
for i in range(test_data.size()[0]) :
    vec = test_data[i]
    label = test_label[i]
    result = 0
    p1 = ldf(vec, mu_1, pi_1, cor_ma)
    p2 = ldf(vec, mu_2, pi_2, cor_ma)
    p3 = ldf(vec, mu_3, pi_3, cor_ma)
    if p1 > p2 :
        if p1 > p3 : result = 1
        else : result = 3
    else :
        if p2 > p3 : result = 2
        else : result = 3
    if result == label :
        right += 1
print(right / (32 - 20))
# 0.75

0.75
